<a href="https://colab.research.google.com/github/Abhishek0697/Detection-of-Hate-Speech-in-Multimodal-Memes/blob/main/Code/Experiments/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Check GPU availability
!nvidia-smi

Thu Mar  4 09:34:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    28W /  70W |   4820MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !pip install transformers~=2.11.0
!pip install transformers~=4.3.3

In [ ]:
'''Importing necessary modules
'''
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('./Trainers/')
sys.path.append('./Dataloaders/')
sys.path.append('./utils/')
sys.path.append('/content/gdrive/MyDrive/Kaggle/Hateful meme detection local files')

from dataloader import mydataset
'''
For BERT
'''
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from trainer_BERT import train, test_classify
from Load_model import load

In [ ]:
# Select gpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


**Dataloading Scheme**

In [ ]:
trainlist = '/content/gdrive/MyDrive/Kaggle/facebook-hateful-meme-dataset/data/train.jsonl'
validlist = '/content/gdrive/MyDrive/Kaggle/facebook-hateful-meme-dataset/data/dev.jsonl'

In [ ]:
# prime_dict = create_prime_dict(trainlist)

In [ ]:
'''
Train Dataloader
''' 
train_dataset = mydataset(trainlist, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 32, num_workers=16,pin_memory=True)


'''
Validation Dataloader
''' 
validation_dataset = mydataset(validlist, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 32, num_workers=16,pin_memory=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


**Model Definition**

In [ ]:
'''
Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
''' 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", #12-layer BERT model, with an uncased vocab.
    num_labels = 2, #Binary classification
    output_attentions = True, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
)

model = nn.DataParallel(model,device_ids=[0]).to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

**Hyperparameters**

In [ ]:
'''
Loss Function
'''
criterion = nn.CrossEntropyLoss()


'''
Optimizer
'''
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

'''
Number of training epochs. The BERT authors recommend between 2 and 4. We chose to run for 4, but we'll see later that this may be over-fitting the training data.
'''
num_Epochs = 10


'''
Create the learning rate scheduler.
Total number of training steps is [number of batches] x [number of epochs].
Note that this is not the same as the number of training samples.
'''
total_steps = len(train_dataloader) * num_Epochs
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0,  num_training_steps = total_steps)

In [ ]:
modelname = 'BERT_basic'
modelpath = '/content/gdrive/MyDrive/Kaggle/saved_model_checkpoints/new/'+modelname

In [ ]:
writer = SummaryWriter(modelname)

train(model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, modelpath, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

loss 0.6178842473030091
loss 1.1973635482788085
Epoch:  1
training loss =  0.5882176616810318
Validation Loss: 0.7871	Top 1 Validation Accuracy: 0.5660


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


loss 0.5155595102906227
loss 1.037861301600933
Epoch:  2
training loss =  0.5131523501604123
Validation Loss: 0.8528	Top 1 Validation Accuracy: 0.5580
loss 0.4101566496491432
loss 0.8612087018787861
Epoch:  3
training loss =  0.43874646135066686
Validation Loss: 0.9157	Top 1 Validation Accuracy: 0.5380
loss 0.3697449783980846
loss 0.7572229799628257
Epoch:  4
training loss =  0.3824126777008064
Validation Loss: 1.0431	Top 1 Validation Accuracy: 0.5340
loss 0.3060436261445284
loss 0.648694257363677
Epoch:  5
training loss =  0.33361895183535445
Validation Loss: 1.2518	Top 1 Validation Accuracy: 0.5300
loss 0.27918849892914294
loss 0.5871433471888303
Epoch:  6
training loss =  0.30002638728434877
Validation Loss: 1.3783	Top 1 Validation Accuracy: 0.5420
loss 0.2599613666534424
loss 0.5435575449466705
Epoch:  7
training loss =  0.2744963364605617
Validation Loss: 1.5002	Top 1 Validation Accuracy: 0.5300
loss 0.2530093207210302
loss 0.5136955760419368
Epoch:  8
training loss =  0.259387723

In [ ]:
'''
Load saved model from checkpoint  #####
'''
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:234: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
model.module.classifier

Linear(in_features=768, out_features=2, bias=True)